<a href="https://colab.research.google.com/github/maurorisonho/fraud-detection-neuromorphic/blob/main/notebooks/01-stdp_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir no Colab"/></a>

# Exemplo STDP: Aprendizado Biológico

**Descrição:** Tutorial Interativo sobre o mecanismo biológico de aprendizado STDP (Plasticidade Dependente do Tempo de Spike) usado em redes neurais neuromórficas. Demonstra como os neurônios aprendem correlações temporais automaticamente.

**Autor:** Mauro Risonho de Paula Assumpção.
**Data de Criação:** 5 Dezembro 2025.
**Licença:** MIT License.
**Desenvolvimento:** Desenvolvimento Humano + Assistido por IA (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# Instalar a biblioteca Brian2 se ainda não estiver instalado
try:
    import brian2
except ImportError:
    !pip install brian2
    import brian2

# Importação específica do brian2 ao invés de wildcard
from brian2 import (
    ms, mV, Hz, second,
    NeuronGroup, Synapses, SpikeMonitor, StateMonitor,
    SpikeGeneratorGroup, Network,
    defaultclock, run, device, start_scope,
    clip, prefs
)

# Configurar para usar numpy (evita erro de compilação C++ se os headers estiverem faltando)
prefs.codegen.target = "numpy"

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✓ Importações concluídas!")


# STDP: Plasticidade Dependente do Tempo de Spike

**Descrição:** Tutorial Interativo sobre o mecanismo biológico de aprendizado STDP (Plasticidade Dependente do Tempo de Spike) usado em redes neurais neuromórficas. Demonstra como os neurônios aprendem correlações temporais automaticamente.

**Autor:** Mauro Risonho de Paula Assumpção
**Data de Criação:** 5 Dezembro 2025
**Licença:** MIT License
**Desenvolvimento:** Desenvolvimento Humano +  Assistido por IA (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

Este notebook explora o mecanismo biológico de aprendizado **STDP** usado em redes neurais neuromórficas.

## O que é STDP?

STDP (Plasticidade Dependente do Tempo de Spike) é uma regra de aprendizado **não supervisionada** inspirada por neurônios biológicos:

- **se o neurônio pré-sináptico dispara ANTES o pós-sináptico** → **Potenciação** (peso ↑)
- **se o neurônio pré-sináptico dispara DEPOIS o pós-sináptico** → **Depressão** (peso ↓)

Isso permite que a rede aprenda **relações causais temporais** sem rótulos explícitos.

## 1. Configuração e Importações

## 1. Curva STDP Clássica

Visualizar como a mudança no peso depende da diferença temporal entre disparos.

In [ ]:
# STDP Parâmetros
tau_pre = 20.0 # ms - constante de tempo pré-sináptico
tau_post = 20.0 # ms - constante de tempo pós-sináptico
A_pre = 0.01 # Amplitude de Potenciação
A_post = -0.012 # Amplitude de Depressão

# Delta t (diferença temporal)
dt_range = np.linspace(-100, 100, 500) # ms

# Calcular mudança de peso
def stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post):
    """
    Calcula a mudança de peso de acordo com STDP.
    dt = t_post - t_pre
    """
    if dt > 0: # Post depois Pre → Potenciação
        return A_pre * np.exp(-dt / tau_pre)
    else: # Post antes Pre → Depressão
        return A_post * np.exp(dt / tau_post)

weight_changes = np.array([stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post)
                           for dt in dt_range])

# Plotar curva STDP
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dt_range, weight_changes, linewidth=3, color='purple')
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Anotar regiões
ax.fill_between(dt_range[dt_range > 0], 0, weight_changes[dt_range > 0],
                alpha=0.2, color='green', label='Potenciação (LTP)')
ax.fill_between(dt_range[dt_range < 0], 0, weight_changes[dt_range < 0],
                alpha=0.2, color='red', label='Depressão (LTD)')

ax.set_xlabel('Δt = t_post - t_pre (ms)', fontsize=12)
ax.set_ylabel('Mudança de peso (Δw)', fontsize=12)
ax.set_title('Curva STDP: Plasticidade Dependente do Tempo de Spike', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Adicionar anotações
ax.annotate('Pré → Pós\n(Causal)', xy=(20, 0.008), fontsize=10,
            ha='center', color='green', fontweight='bold')
ax.annotate('Pós → Pré\n(Anti-causal)', xy=(-20, -0.009), fontsize=10,
            ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Interpretação:")
print(" - Δt > 0: neurônio pré dispara ANTES → Potenciação (reforça conexão)")
print(" - Δt < 0: neurônio pré dispara DEPOIS → Depressão (enfraquece conexão)")
print(" - Efeito decai exponencialmente com |Δt|")


## 2. Simulação STDP com Brian2

Simular dois neurônios conectados com STDP e observar evolução dos pesos.

In [ ]:
start_scope()

# Parâmetros da Simulação
duration = 100*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ Configurando Simulação STDP...")
print(f"Duração: {duration}")
print(f"Passo de tempo: {defaultclock.dt}\n")

# neurônios LIF
tau_m = 10*ms  # type: ignore[operator]
tau_syn = 5*ms  # type: ignore[operator] # Constante de tempo do sinapse
v_rest = -70*mV
v_thresh = -50*mV
v_reset = -70*mV

# Adicionado decaimento sináptico (dI_syn/dt)
eqs_post = '''
dv/dt = (v_rest - v + I_syn) / tau_m : volt
dI_syn/dt = -I_syn / tau_syn : volt
'''

# Criar neurônios
neuron_pre = SpikeGeneratorGroup(1, [0], [10]*ms)  # type: ignore[operator]
neuron_post = NeuronGroup(1, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest
neuron_post.I_syn = 0*mV

# STDP Parâmetros
tau_pre_stdp = 20*ms  # type: ignore[operator]
tau_post_stdp = 20*ms  # type: ignore[operator]
A_pre_stdp = 0.01
A_post_stdp = -0.012
w_max = 1.0
w_min = 0.0

synapse_model = '''
w : 1
dApre/dt = -Apre / tau_pre_stdp : 1 (event-driven)
dApost/dt = -Apost / tau_post_stdp : 1 (event-driven)
'''

# Aumentado ganho sináptico para garantir disparo (w * 60*mV)
on_pre_stdp = '''
I_syn_post += w * 60 * mV
Apre += A_pre_stdp
w = clip(w + Apost, w_min, w_max)
'''

on_post_stdp = '''
Apost += A_post_stdp
w = clip(w + Apre, w_min, w_max)
'''

synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect(i=0, j=0)
synapse.w = 0.5 # peso inicial

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)
mon_voltage = StateMonitor(neuron_post, 'v', record=True)

# Executar Simulação
print("⏳ Executando Simulação Brian2...")
import time
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight, mon_voltage)
net.run(duração)

sim_time = time.time() - start_time

print(f" Simulation concluída em {sim_time:.3f}s!")
print(f"\n Resultados:")
print(f" Disparos pré-sinápticos: {len(mon_pre.t)}")
print(f" Disparos post-synaptics: {len(mon_post.t)}")
print(f" peso inicial: {0.5:.3f}")
print(f" peso final: {mon_weight.w[0][-1]:.3f}")
print(f" Mudança: {(mon_weight.w[0][-1] - 0.5):.3f} ({(mon_weight.w[0][-1] - 0.5)/0.5*100:+.1f}%)")

In [ ]:
# Visualizar results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Disparos
if len(mon_pre.t) > 0:
    axes[0].eventplot([mon_pre.t/ms], lineoffsets=1, linelengths=0.8,
                      linewidths=2, colors='blue', label='pre-synaptic')
if len(mon_post.t) > 0:
    axes[0].eventplot([mon_post.t/ms], lineoffsets=0, linelengths=0.8,
                      linewidths=2, colors='red', label='post-synaptic')

axes[0].set_ylabel('neuron')
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['Pós', 'Pré'])
axes[0].set_title('Raster Plot: Disparos Pré e Pós-Synaptics', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Plot 2: Evolução do peso sináptico
axes[1].plot(mon_weight.t/ms, mon_weight.w[0], linewidth=2.5, color='purple')
axes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='weight Inicial')
axes[1].set_ylabel('weight Synaptic (w)')
axes[1].set_title('Evolução do Synaptic peso com STDP', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Voltagem do neurônio pós-sináptico
axes[2].plot(mon_voltage.t/ms, mon_voltage.v[0]/mV, linewidth=1.5, color='green')
axes[2].axhline(-50, color='red', linestyle='--', alpha=0.7, label='Threshold')
axes[2].axhline(-70, color='gray', linestyle='--', alpha=0.5, label='Resting')
axes[2].set_xlabel('time (ms)')
axes[2].set_ylabel('Voltagem (mV)')
axes[2].set_title('Potencial de Membrana pós-sináptico', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. STDP com padrões de entrada

Demonstrar como STDP aprende correlações temporais em padrões repetidos.

In [ ]:
start_scope()

# Simular multiple neurônios pré-sinápticos
n_pre = 5
n_post = 1
duração = 500*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print(" Configurando Simulation com multiple neurônios...")
print(f"neurônios pré-sinápticos: {n_pre}")
print(f"Duração: {duração}\n")

# Gerar padrão temporal (some neurônios disparar em sequence)
spike_pattern = [
    [10, 110, 210, 310, 410], # neurônio 0: disparos regulares
    [15, 115, 215, 315, 415], # neurônio 1: levemente atrasado
    [20, 120, 220, 320, 420], # neurônio 2: more atrasado
    [100, 200, 300, 400], # neurônio 3: disparos esparsos
    [50, 150, 250, 350, 450] # neurônio 4: fase different
]

indices = []
times = []
print(" patterns de disparos:")
for neuron_idx, disparo_times em enumerate(spike_pattern):
    print(f" neurônio {neuron_idx}: {len(spike_times)} disparos")
    para t em disparo_times:
        indices.append(neuron_idx)
        times.append(t)

neuron_pre = SpikeGeneratorGroup(n_pre, indices, times*ms)  # type: ignore[operator]

# neurônio pós-sináptico
neuron_post = NeuronGroup(n_post, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest

# Sinapses com STDP
synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect() # Conectar all
synapse.w = 'rand() * 0.3 + 0.2' # pesos initial aleatórios [0.2, 0.5]

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)

# Save pesos initial
initial_pesos = np.array(synapse.w).copy()

# Executar
print("\n⏳ Executing Simulation de temporal patterns...")
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight)
net.run(duração)

sim_time = time.time() - start_time
final_pesos = np.array(synapse.w).copy()

print(f" Simulation concluída em {sim_time:.3f}s!")
print(f"\n Analysis de o pesos Synaptics:")
for i em range(n_pre):
    delta = final_pesos[i] - initial_pesos[i]
    percentage = (delta / initial_pesos[i]) * 100 if initial_pesos[i] > 0 else 0
    print(f" neurônio {i}: {initial_pesos[i]:.3f} → {final_pesos[i]:.3f} (Δ = {delta:+.3f}, {percentage:+.1f}%)")

In [ ]:
# Visualizar evolução dos pesos
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Temporal evolução dos pesos
for i em range(n_pre):
 axes[0].plot(mon_weight.t/ms, mon_weight.w[i], label=f'Synapse {i}', linewidth=2)

axes[0].set_xlabel('time (ms)')
axes[0].set_ylabel('weight Synaptic')
axes[0].set_title('Evolução Temporal de Pesos Sinápticos com STDP', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Plot 2: Comparação before/after
x_pos = np.arange(n_pre)
width = 0.35

axes[1].bar(x_pos - width/2, initial_pesos, width, label='Inicial', alpha=0.7, color='lightblue')
axes[1].bar(x_pos + width/2, final_pesos, width, label='Final', alpha=0.7, color='darkblue')

axes[1].set_xlabel('neuron pre-synaptic')
axes[1].set_ylabel('weight Synaptic')
axes[1].set_title('Comparison: pesos Initial vs Finais', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n Interpretation:")
print(" - neurônios that disparar consistentemente ANTES de o post-synaptic são reinforced")
print(" - neurônios com timing inconsistente têm pesos reduzidos")
print(" - A network aprende a temporal correlation automatically!")

## 4. Aplicação para a Detecção de Fraude

como STDP ajuda na detecção de fraude?

### Scenario 1: Sequência Temporal Normal

**Transação Legítima:**
1. Login no aplicativo (t=0ms)
2. Navegação no saldo (t=500ms)
3. Seleção de beneficiário conhecido (t=2000ms)
4. Confirmação de pagamento (t=3000ms)

**STDP aprende:**
- Sequência causal esperada
- Intervalos temporais normais
- Reforça conexões que representam comportamento legítimo

### Scenario 2: Sequência Anômala (Fraude)

**Transação Fraudulenta:**
1. Login no aplicativo (t=0ms)
2. Transferência imediata sem navegação (t=50ms)
3. alto valor para novo beneficiário (t=100ms)
4. Localização geográfica inconsistente (t=150ms)

**STDP detecta:**
- padrão temporal anomalous
- Sequência não reforçada durante Treinamento
- alto activation de neurônios de "fraude"

### Vantagens do STDP:

1. **Learning unsupervised**: não necessita de rótulos explícitos inicialmente
2. **Continuous adaptation**: Learns novo fraude patterns automatically
3. **Temporal sensitivity**: Detecta anomalias na sequência de eventos
4. **eficiência**: Atualização local de peso (sem retropropagação)
5. **Biologically plausible**: Inspirado no cérebro humano

## 5. Conclusões

### STDP em Detection de Fraude

**mecanismo:**
- Learns correlações temporais entre features de transaction
- Reforça padrões legítimos frequentes
- Detecta desvios na sequência temporal

**Aplicações Práticas:**
1. **Análise de behavior**: Sequência de ações no banco móvel
2. **Detection de speed**: Transações impossíveis (ex: compras em cidades diferentes em poucos minutos)
3. **patterns de usage**: Horários, frequência, valores típicos
4. **Suspicious navigation**: Sequências atípicas de páginas

**Comparação com métodos tradicionais:**

| característica | STDP/SNN | DNN/LSTM |
|----------------|----------|----------|
| Processamento temporal | Nativo | Emulado |
| Supervisão | not | yes |
| latência | Ultra-low (~ms) | alto (~100ms) |
| consumo energético | muito baixo | alto |
| Adaptação online | yes | Difficult |
| Hardware especializado | yes (Loihi, TrueNorth) | GPU |

### Futuro

- Chips neuromórficos dedicados (Intel Loihi 2, IBM NorthPole)
- STDP + Reward modulation (dopamina artificial)
- Learning federado com STDP
- Explicabilidade: Visualizar pesos learned

---

**Autor:** Mauro Risonho de Paula Assumpção
**Projeto:** Computation Neuromórfica para Cybersecurity Banking